# Classification Exercise

This notebook is based on the notebook from [Chapter 3](https://github.com/ageron/handson-ml3/blob/main/03_classification.ipynb) of the Scikit-learn textbook.

Last week we looked at a regression task, where we predicted the median house price of a district based on various features. **Classification** is another common supervised machine learning problem where we predict a categorical label, such as whether a picture is of a cat or a potato. In this notebook, we'll look at the classic MNIST dataset, which is technically an image classification problem, but the images are so tiny that we can treat each pixel as a feature.

This dataset was used to train the one of the first practical neural networks in 1989 - identifying handwritten digits in US zip codes.

## Goals
The main objective of this exercise is to:
- Understand the differences between regression and classification tasks
- Learn about metrics for evaluating classification models
- Gain exposure to some more Scikit-learn tools

## Load and explore the data

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False, parser="auto")
# Read some info about MNIST, including a reference to the original source.
print(mnist.DESCR)

In [ ]:
# mnist is a dictionary-like object. We only really need the data and target.
X, y = mnist.data, mnist.target 

# 28x28 pixels per image, 70,000 images total.
print(X.shape, y.shape)

# be careful - the labels are strings, not numbers!
y[:10]

In [ ]:
import matplotlib.pyplot as plt

# plot a few images and their labels
# looks like numbers!
fig, axes = plt.subplots(2, 5, figsize=(8,4))
for i, ax in enumerate(axes.flat):
    ax.imshow(X[i].reshape(28, 28), cmap='gray_r')
    ax.set(title = f"Label: {y[i]}")
plt.show()

In [ ]:
# split into train and test sets - 60,000 for training, 10,000 for testing
# Note: we could use train_test_split here, but we'll do it manually because that's what the book does.
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

# make sure the distribution of labels in the train and test sets is similar
import numpy as np

y_train_counts = np.unique(y_train, return_counts=True)
y_test_counts = np.unique(y_test, return_counts=True)

plt.bar(y_train_counts[0], y_train_counts[1], color='blue', label='train')
plt.bar(y_test_counts[0], y_test_counts[1], color='orange', label='test')
plt.legend()
plt.xlabel("Label (digit)")
plt.ylabel("Count")

This is about as clean as a dataset gets - can you think of any preprocessing that might be useful?

## Train a model
For the regression example we spent a bunch of time on preprocessing and data exploration, but this time we'll dive in to classification as I want to get to the metrics.

The model we'll use is the [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html), which implements some kind of linear classifier using stochastic gradient descent. With default parameters, it trains a support vector machine (SVM), while the "stochastic" part means that the gradient is computed on a single random sample at a time. It's fast as a result, and can be used for large datasets, but might not be the most accurate.

The SGDClassifier also includes a penalty term or regularization parameter, which imposes a penalty on the weights and helps to avoid overfitting. By default the penalty term is the L2 norm, or the sum of the squares of the weights. You can read more about the math going on [here](https://scikit-learn.org/stable/modules/sgd.html#mathematical-formulation).

We'll also start with a binary classification problem, arbitrarily choosing the digit 5. Many of the metrics we'll look at are most intuitive for binary classification.

In [ ]:
# Extract a binary subset of "5 or not 5" 
# True for all 5s, False for all other digits
y_train_5 = (y_train == '5')  
y_test_5 = (y_test == '5')

# train a binary classifier
from sklearn.linear_model import SGDClassifier

# Use the defaults, but understand what that implies
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

In [ ]:
# Choose some random samples to predict
some_digits = X_train[:10]
print(y_train_5[:10])
sgd_clf.predict(some_digits) # looks pretty good!


From this spot check, it looks like the classifier is doing a pretty good job - there's one 5 followed by a bunch of not-5s. But how should we actually measure the performance? Like the regression task, let's try cross-validation with **accuracy** as the metric.

## Terminology
- **True positive**: the model predicted positive and the label was positive ($TP$). 
- **True negative**: the model predicted negative and the label was negative ($TN$). 
- **False positive**: the model predicted positive but the label was negative ($FP$). Also called **Type I error**.
- **False negative**: the model predicted negative but the label was positive ($FN$). Also called **Type II error**.

## Accuracy
Accuracy is the fraction of correct predictions, given as:

$$\mathrm{accuracy} = \frac{TP + TN}{TP + TN + FP + FN}$$

It's pretty intuitive - out of all the predictions, how many were correct? Of course, we know by now not to just look at the accuracy of the training set, so let's do some cross-validation.

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")

Looks pretty solid! However, is accuracy the best metric in this scenario? Let's define a new classifier that just says "everything is not a 5" and see what happens.

In [ ]:
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        return self
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)
    
never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring="accuracy")

Oh look, 90% accuracy just by saying `False` all the time. We need to be really careful with accuracy as a metric, especially when the classes are **imbalanced**. We have about 10% 5s and 90% not-5s. Going back to our formula and plugging in some numbers, we get:

$$TP = 0$$
$$TN = 90$$
$$FP = 10$$
$$FN = 0$$
$$\mathrm{accuracy} = \frac{0 + 90}{0 + 90 + 10 + 0} = 0.9$$

The math works. Clearly we need something better than accuracy.

## More Terminology
Honestly, this stuff gets pretty confusing and it can be hard to remember which is which. The [Wikipedia page](https://en.wikipedia.org/wiki/Precision_and_recall) is pretty good, but I'll try to summarize here.

- **Precision**: Out of all the positive **predictions**, how many were correct? Given as:
  $$\mathrm{precision} = \frac{TP}{TP + FP}$$
  Precision is also called the **positive predictive value**.
- **Recall**: Out of all the positive **labels**, how many were correctly predicted? Given as:
  $$\mathrm{recall} = \frac{TP}{TP + FN}$$
  Recall is also called the **sensitivity** or **true positive rate**.
- **Specificity**: Out of all the negative **labels**, how many were correctly predicted? Given as:
  $$\mathrm{specificity} = \frac{TN}{TN + FP}$$
  Specificity is also called the **true negative rate**.

There are more combinations of these terms, but these are the ones you'll see most often. Let's begin with a **confusion matrix**, which is a table of the true labels vs. the predicted labels.

## Confusion Matrix

| | Predicted Positive | Predicted Negative |
| --- | --- | --- |
| **True Positive** | TP | FN |
| **True Negative** | FP | TN |

An excellent classifier will have strong diagonal elements, meaning that the true labels and predicted labels agree. This is a great visual way to see how your classifier is doing, but we need to calculate metrics to actually summarize the performance.

Note that scikit-learn flips the axes such that the confusion matrix is:

| | Predicted Negative | Predicted Positive |
| --- | --- | --- |
| **True Negative** | TN | FP |
| **True Positive** | FN | TP |

Ultimately it doesn't matter as long as you understand what's going on.

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, sgd_clf.predict(X_train))
# Not too bad, the diagonal elements are definitely stronger than the off-diagonal.
# Still a lot of false positives though, which we expect from our mostly not-5 dataset.

In [ ]:
# we can also visualize it as a prettier heatmap
from sklearn.metrics import ConfusionMatrixDisplay
cm = confusion_matrix(y_train_5, sgd_clf.predict(X_train))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['not 5', '5'])
disp.plot()

## ✔️ Precision and Recall
**Exercise**:

Using the provided formulas and the confusion matrix shown above, calculate the precision and recall for this classifier manually. Then use the `precision_score` and `recall_score` functions to check your work.

In [ ]:
from sklearn.metrics import precision_score, recall_score
# Do your manual calculations, then compare to the built-in functions

## Precision-Recall Tradeoff
It would be great to be able to improve both precision and recall, but usually you have to decide which is more important. Internally, the `SGDClassifier` computes a score for each instance, then applies a **threshold** to decide whether it's positive (`'5'`) or negative (`'not-5'`). We can't set the threshold parameter directly, but we can look at the scores and apply our own threshold.

In [ ]:
import numpy as np
jitter = 0.01 # add some jitter to make the plot easier to read
scores = sgd_clf.decision_function(X_train)
new_thresh = -5000
plt.scatter(y_train_5 + jitter * np.random.randn(len(y_train_5)), scores, alpha=0.1)
plt.xlabel("Label (0 = not 5, 1 = 5)")
plt.ylabel("Decision function score")
plt.plot([-0.1, 1.1], [0, 0], 'k--', label="Default threshold")
plt.plot([-0.1, 1.1], [new_thresh, new_thresh], 'r--', label="Higher TP threshold")
plt.legend()

With the new threshold of -5000, we're increasing the number of true positives - but we're also capturing a whole lot of false positives. This is the **precision-recall tradeoff**. If we calculate the numbers for the new threshold, we get:

In [ ]:
new_y = (scores > new_thresh)
print("Precision:", precision_score(y_train_5, new_y))
print("Recall:", recall_score(y_train_5, new_y))

Excellent recall, but our precision is terrible! Which threshold should we use? It depends on the application and what is more important. For example, if you're designing a classifier to detect Covid, you might want to err on the side of false positives (low precision) to avoid missing any cases (high recall).

## F1 Score
A common way to combine the precision and recall into a single metric is the [$F_1$ score](https://en.wikipedia.org/wiki/F-score), or the harmonic mean of the two:

$$F_1 = \frac{2}{\frac{1}{\mathrm{precision}} + \frac{1}{\mathrm{recall}}} = 2 \times \frac{\mathrm{precision} \times \mathrm{recall}}{\mathrm{precision} + \mathrm{recall}} = \frac{2TP}{2TP + FP + FN}$$

Maximizing the $F_1$ score is a way to maximize both precision and recall at the same time, but again, this isn't always what you want. There's also $F_\beta$ score, which is a weighted average of precision and recall, where $\beta$ is a parameter that controls the weight:

$$F_\beta = (1 + \beta^2) \times \frac{\mathrm{precision} \times \mathrm{recall}}{\beta^2 \times \mathrm{precision} + \mathrm{recall}} = (1 + \beta^2) \times \frac{TP}{(1 + \beta^2)TP + \beta^2FN + FP}$$

When $\beta = 1$, we get the $F_1$ score. When $\beta < 1$, we're giving more weight to precision, and when $\beta > 1$, we're giving more weight to recall.

## ROC Curve
ROC stands for [receiver operating characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). It's a plot of the **true positive rate** (recall or sensitivity) vs. the **false positive rate** (1 - specificity) as our threshold changes. We want both sensitivity and specificity to be as high as possible, so the ideal threshold is somewhere in the upper left corner of the plot.

A line of $TPR = FPR$ (the diagonal) represents random guessing, while a perfect classifier would hug the top left corner. It's common to see ROC curves with the diagonal drawn in as a reference.

> Fun fact: the name "ROC" comes from its development during World War II to analyze radar signals. A true positive is a radar blip corresponding to an actual aircraft, while a false positive corresponds to radar noise.

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train_5, scores)

# plot the ROC curve
plt.plot(fpr, tpr, linewidth=2)
plt.plot([0,1], [0,1], 'k--')
plt.grid(True)
plt.xlabel("False positive rate (1 - specificity)")
plt.ylabel("True positive rate (sensitivity)")

# label the default threshold of 0
zero_thresh_i = np.argmin(abs(thresholds))
plt.plot(fpr[zero_thresh_i], tpr[zero_thresh_i], 'ro', label="Default threshold")

# And the new one at -5000
new_thresh_i = np.argmin(abs(thresholds - new_thresh))
plt.plot(fpr[new_thresh_i], tpr[new_thresh_i], 'go', label="Higher TP threshold")
plt.legend()

## Area under the ROC Curve (AUC)
The ROC curve can be useful for choosing a threshold,  but it can also be used to compare different classifiers. The **area under the curve** (AUC) is a single number that summarizes the performance of the classifier. Since our plot is normalized to a range of 0-1 for both axes, a perfect classifier would have an AUC of 1, while a random guess would be 0.5. If the AUC is less than 0.5, something is very wrong, or you might want to invert the predictions.

We need another classifier to compare to, so let's add on a `RandomForestClassifier`. The next cell uses a cross-validation with $k=3$ folds, so it may take a minute or so to run.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

forest_clf = RandomForestClassifier(random_state=42)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3, method="predict_proba")

y_scores_forest = y_probas_forest[:, 1]
# score = proba of positive class
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5,y_scores_forest)

In [ ]:
# calculate the AUC for both classifiers
from sklearn.metrics import roc_auc_score
sgd_auc = roc_auc_score(y_train_5, scores)
forest_auc = roc_auc_score(y_train_5, y_scores_forest)

# Plot both ROC curves on the same plot
plt.plot(fpr, tpr, linewidth=2, label=f"SGD: AUC={sgd_auc:.3f}")
plt.plot(fpr_forest, tpr_forest, linewidth=2, label=f"Random Forest: AUC={forest_auc:.3f}")
plt.plot([0,1], [0,1], 'k--')
plt.grid(True)
plt.xlabel("False positive rate (1 - specificity)")
plt.ylabel("True positive rate (sensitivity)")
plt.legend()

## ✔️ Exercises
Feel free to insert cells as needed to answer the following questions.

1. Given the following two confusion matrices:
    ```python
    c_matrix_classifier1 = np.array([[200, 100], [100, 600]])
    c_matrix_classifier2 = np.array([[170, 130], [50, 650]])
    ```
    1. Calculate the precision, recall, and F1 score for each classifier.
    
    Which classifier would you choose if you were trying to predict:
    
    2. Whether a video is safe for kids to watch?
    3. Whether a credit card transaction is fraudulent?

    Justify your answers.

    > Think about the consequences of false positives and false negatives for each scenario. It might help to plot the confusion matrices to visualize the results. Note that these are defined in the scikit-learn convention, where the top left is true negatives and the bottom right is true positives.

2. Given the following ROC curves for two classifiers: 

    ![ROC Curves](fake_roc.png)

    Without knowing anything about the context:
    1. Which classifier is better overall?
    2. What are the approximate precision and recall values corresponding to the maximum $F_1$ score for each classifier?
        > Hint: The maximum $F_1$ score corresponds to the point on the curve in the top-left corner.
    3. Is there ever a scenario where you would choose the weaker classifier?

# Extras
The main thing I wanted to cover in this notebook is the binary classification + metrics, but if you're interested in extending to multi-class classification, read on.

## Multi-class Classification
So far most of the metrics discussed are only applicable to binary classification. However, many tasks (such as the digit recognition problem) are actually **multi-class** classification problems. There are three basic strategies to handle this:
1. **One-versus-all** (OvA): Train a binary classifier for each class, then choose the class with the highest score.
2. **One-versus-one** (OvO): Train a binary classifier for each pair of classes, then choose the class that wins the most pairs.
3. Use a classifier that can handle multiple classes directly, such as a Random Forest or Naive Bayes classifier.

Let's try a Random Forest classifier on the MNIST dataset, and then look at the confusion matrix.

In [ ]:
# As usual random forests take a bit of time to train, especially with cross validation
multi_forest = RandomForestClassifier(random_state=42)
multi_forest.fit(X_train, y_train)
cv_predictions = cross_val_predict(multi_forest, X_train, y_train, cv=3)

In [ ]:
cm_forest = confusion_matrix(y_train, cv_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_forest, display_labels=multi_forest.classes_)
disp.plot()
# Looks pretty great!

Finally, let's look at the precision and recall for each class. These are still just binary metrics, but we can go one class at a time and calculate:

$$\mathrm{precision_{class}} = \frac{TP_{class}}{TP_{class} + FP_{class}}$$

where $TP_{class}$ is the element on the diagonal of the confusion matrix, $FP_{class}$ is the sum of the elements in the column, and $FN_{class}$ is the sum of the elements in the row. For example, the precision for class 0 is:

$$\mathrm{precision_{0}} = \frac{5840}{5840 + 1 + 27 + 7 + 12 + 20 + 26 + 4 + 9 + 21} = 0.979$$

In [ ]:
# Calculate the precision and recall for each class
multi_precision = precision_score(y_train, cv_predictions, average=None)
multi_recall = recall_score(y_train, cv_predictions, average=None)

plt.boxplot([multi_precision, multi_recall], labels=['Precision', 'Recall'])
plt.ylabel("Score")
plt.ylabel("Metric")
plt.title("Average Precision and Recall across all classes")

# and the mean F1 score to summarize
from sklearn.metrics import f1_score
mean_f1 = np.mean(f1_score(y_train, cv_predictions, average=None))
print(f"Mean F1 score: {mean_f1:.3f}")

In [ ]:
# Finally, let's see how it behaves on the test set
test_predictions = multi_forest.predict(X_test)
cm_test = confusion_matrix(y_test, test_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=multi_forest.classes_)
disp.plot() # still looking good!

In [ ]:
# This cell just generates figure with fake ROC data for the exercises above
import numpy as np

# Define the amount of jitter
jitter = 0.02

# Generate smooth curves
fake_fpr = np.geomspace(1, 2, 100) - 1
tpr1 = fake_fpr ** 0.1
tpr2 = fake_fpr ** 0.2

# Add jitter, because it's usually not perfectly smooth
tpr1 += jitter * np.random.rand(len(tpr1)) - jitter / 2
tpr2 += jitter * np.random.rand(len(tpr2)) - jitter / 2

plt.plot(fake_fpr, tpr1, linewidth=2, label="Classifier 1")
plt.plot(fake_fpr, tpr2, linewidth=2, label="Classifier 2")
plt.plot([0, 1], [0, 1], "k--")
plt.grid(True)
plt.xlabel("False positive rate (1 - specificity)")
plt.ylabel("True positive rate (sensitivity)")
plt.legend()

plt.savefig("fake_roc.png")